# Ceneo Scraper
**Struktura opinii w serwisie ceneo.pl:**
|Składowa|Selektor|Zmienna|
|--------|--------|--------|
|*identyfikator opinii*|["data-entry-id"]|`opinion_id`|
|*autora*|span.user-post__author-name|`author`|
|*rekomendację*|span.user-post__author-recomendation > em|`recommend`|
|*liczbę gwiazdek*|span.user-post__score-count|`stars`|
|*treść opinii*|div.user-post__text|`content`|
|*listę wad*|div.review-feature__title review-feature__title--negatives ~ div.review-feature__item|`cons`|
|*listę zalet*|div.review-feature__title review-feature__title--positives ~ div.review-feature__item|`pros`|
|*data wystawienia opinii*|span.user-post__published > time:nth-child(1)["datetime"]|`opinion_date`|
|*data zakupu produktu*|span.user-post__published > time:nth-child(2)["datetime"]|`purchase_date`|
|*ile osób uznało opinię za przydatną*|button.vote-yes["data-total-vote"]|`upvote`|
|*ile osób uznało opinię za nieprzydatną*|button.vote-no["data-total-vote"]|`downvote`|



In [118]:
selectors = {
    "opinion_id": (None, "data-entry-id"),
    "author" : ("span.user-post__author-name",),
    "recommend" : ("span.user-post__author-recomendation > em",),
    "stars" : ("span.user-post__score-count",),
    "content": ("div.user-post__text",),
    "cons": ("div.review-feature__title--negatives ~ div.review-feature__item", None, True),
    "pros": ("div.review-feature__title--positives ~ div.review-feature__item", None, True),
    "opinion_date" : (("span.user-post__published > time:nth-child(1)"),"datetime"),
    "purchased_date" : (("span.user-post__published > time:nth-child(2)"),"datetime"),
    "upvote": ("button.vote-yes", "data-total-vote"),
    "downvote": ("button.vote-no", "data-total-vote")
}

### Funkcja do ekstrakcji danych z kodu HTML

In [119]:
def extract(ancestor, selector=None, attribute=None, return_list=False):
    if return_list:
        if attribute:
            return [tag[attribute].strip() for tag in ancestor.select(selector)]
        return [tag.text.strip() for tag in ancestor.select(selector)]
    if selector:
        if attribute:
            try:
                return ancestor.select_one(selector)[attribute].strip()
            except TypeError:
                return None        
        try:
            return ancestor.select_one(selector).text.strip()
        except AttributeError:
            return None
    if attribute:
        return ancestor[attribute].strip()
    return ancestor.text.strip()

### 1. Import bibliotek


In [120]:
import os
import json
import requests
from bs4 import BeautifulSoup

### 2. Wczytanie kodu produktu o którym pobierzemy opinie


In [121]:
# product_id = 78035068
product_id = input("Podaj kod produktu: ")
url = f"https://www.ceneo.pl/{product_id}#tab=reviews"

### 3. Pobieranie wszystkich opini o wskazanym produkcie z serwisu Ceneo.pl

In [122]:
all_opinions = []
while(url):
    response = requests.get(url)
    page = BeautifulSoup(response.text, "html.parser")
    opinions = page.select("div.js_product-review")
    for opinion in opinions:
        single_opinion = {
            key: extract(opinion, *value)
                for key, value in selectors.items()
        }
        all_opinions.append(single_opinion)
    try:    
        url = "https://www.ceneo.pl" + extract(page, "a.pagination__next", "href")
    except TypeError:
        url = None

### 4. Zapis opinii o produkcie do pliku JSON do katalogu opinions

In [123]:
if not os.path.exists('opinions'):
    os.mkdir('opinions')
with open(f'opinions/{product_id}.json', "w", encoding="UTF-8") as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False)

all_opinions

[{'opinion_id': '14489974',
  'author': 'm...a',
  'recommend': 'Polecam',
  'stars': '5/5',
  'content': 'Super kryjący krem. Fizyczne filtry, w sam raz dla atopika. Po nałożeniu zostawia bardzo wyraźny biały ślad, który zanika dopiero po kwadransie. Mi to akurat nie przeszkadza. Ciekawy produkt.',
  'cons': [],
  'pros': [],
  'opinion_date': '2021-05-20 21:43:35',
  'purchased_date': '2021-05-08 08:56:45',
  'upvote': '2',
  'downvote': '0'},
 {'opinion_id': '12342496',
  'author': 'Użytkownik Ceneo',
  'recommend': None,
  'stars': '3/5',
  'content': 'Krem ma chyba najlepszy skład na rynku. Same naturalne, mineralne składniki. Z tym, że niestety przez to strasznie trudno się rozsmarowuję i zostawia biały nalot. Przy małym dziecku, które nie lubi smarowania jest to spory problem....',
  'cons': [],
  'pros': [],
  'opinion_date': '2020-05-05 18:58:12',
  'purchased_date': '2020-04-05 00:45:58',
  'upvote': '6',
  'downvote': '1'},
 {'opinion_id': '14668134',
  'author': 's...8',
  